In [ ]:
from django import forms
from django.contrib.auth.forms import UserCreationForm, AuthenticationForm, PasswordResetForm
from django.contrib.auth.tokens import default_token_generator
from django.contrib.sites.shortcuts import get_current_site
from django.core.mail import EmailMessage
from django.template.loader import render_to_string
from django.utils.encoding import force_bytes
from django.utils.http import urlsafe_base64_encode
from django.utils.translation import gettext_lazy as _
from .models import CustomUser


class SignUpForm(UserCreationForm):
    email = forms.EmailField(max_length=254, help_text=_('Required. Inform a valid email address.'))
    first_name = forms.CharField(max_length=30, label=_('First Name'))
    last_name = forms.CharField(max_length=30, label=_('Last Name'))

    class Meta:
        model = CustomUser
        fields = ('username', 'email', 'first_name', 'last_name', 'password1', 'password2')

    def clean_email(self):
        email = self.cleaned_data['email']
        if CustomUser.objects.filter(email=email).exists():
            raise forms.ValidationError(_('This email address is already in use. Please provide a different one.'))
        return email


class UserLoginForm(AuthenticationForm):
    class Meta:
        model = CustomUser
        fields = ('username', 'password')


class CustomPasswordResetForm(PasswordResetForm):
    def send_mail(self, subject_template_name, email_template_name, context, from_email, to_email, html_email_template_name=None):
        """
        Send a django.core.mail.EmailMultiAlternatives to `to_email`.
        """
        subject = render_to_string(subject_template_name, context)
        # Email subject *must not* contain newlines
        subject = ''.join(subject.splitlines())
        body = render_to_string(email_template_name, context)

        email_message = EmailMessage(subject, body, from_email, [to_email])
        if html_email_template_name is not None:
            email_message.attach_alternative(render_to_string(html_email_template_name, context), 'text/html')

        email_message.send()

    def save(self, domain_override=None, subject_template_name='registration/password_reset_subject.txt', 
             email_template_name='registration/password_reset_email.html', use_https=False, token_generator=default_token_generator,
             from_email=None, request=None, html_email_template_name=None, extra_email_context=None):
        """
        Generate a one-use only link for resetting password and send it to the user.
        """
        email = self.cleaned_data["email"]
        for user in self.get_users(email):
            if not domain_override:
                current_site = get_current_site(request)
                site_name = current_site.name
                domain = current_site.domain
            else:
                site_name = domain = domain_override
            context = {
                'email': email,
                'domain': domain,
                'site_name': site_name,
                'uid': urlsafe_base64_encode(force_bytes(user.pk)),
                'user': user,
                'token': token_generator.make_token(user),
                'protocol': 'https' if use_https else 'http',
                **(extra_email_context or {}),
            }
            self.send_mail(
                subject_template_name, email_template_name, context, from_email, email, html_email_template_name=html_email_template_name,
            )


In [ ]:
from django import forms
from .models import Client

class ClientForm(forms.ModelForm):
    secondary_email = forms.EmailField(
        label="Secondary Email Address",
        required=False,
        help_text="Optional: Enter a backup email address.",
        widget=forms.EmailInput(attrs={'class': 'form-control'})
    )
    secondary_phone_number = forms.CharField(
        label="Secondary Phone Number",
        max_length=15,
        required=False,
        widget=forms.TextInput(attrs={'class': 'form-control', 'type': 'tel'}),
    )
    linkedin_profile = forms.URLField(
        label="LinkedIn Profile URL",
        required=False,
        help_text="Optional: Enter your LinkedIn profile URL.",
        widget=forms.URLInput(attrs={'class': 'form-control'})
    )
    twitter_handle = forms.CharField(
        label="Twitter Handle",
        max_length=50,
        required=False,
        help_text="Optional: Enter your Twitter handle.",
        widget=forms.TextInput(attrs={'class': 'form-control'})
    )
    industry = forms.CharField(
        label="Industry",
        max_length=50,
        required=True,
        help_text="Enter the industry the client belongs to.",
        widget=forms.TextInput(attrs={'class': 'form-control'})
    )
    company_size = forms.IntegerField(
        label="Company Size (Number of Employees)",
        min_value=1,
        required=True,
        help_text="Enter the number of employees in the company.",
        widget=forms.NumberInput(attrs={'class': 'form-control'})
    )

    class Meta:
        model = Client
        fields = (
            'secondary_email',
            'secondary_phone_number',
            'linkedin_profile',
            'twitter_handle',
            'industry',
            'company_size',
            # Add other fields here if needed
        )

    def clean_secondary_phone_number(self):
        phone_number = self.cleaned_data['secondary_phone_number']
        # Implement custom validation if needed
        return phone_number


In [ ]:
from django import forms
from django.core.exceptions import ValidationError
from .models import Project

MAX_TITLE_LENGTH = 100
MAX_DESCRIPTION_LENGTH = 500
MAX_NOTES_LENGTH = 1000


class ProjectForm(forms.ModelForm):
    """
    Form for creating or editing projects.
    """

    class Meta:
        model = Project
        fields = [
            "title",
            "description",
            "start_date",
            "end_date",
            "client",
            "team_members",
            "budget",
            "actual_cost",
            "status",
            "priority",
            "is_billable",
            "notes",
        ]
        labels = {
            "start_date": "Start Date",
            "end_date": "End Date",
            "team_members": "Team Members",
            "actual_cost": "Actual Cost",
            "is_billable": "Is Billable",
        }
        help_texts = {
            "title": "Enter a descriptive title for the project.",
            "budget": "Specify the budget allocated for the project.",
            "status": "Select the current status of the project.",
            "priority": "Set the priority level of the project.",
        }
        widgets = {
            "notes": forms.Textarea(attrs={"rows": 3}),
            "team_members": forms.SelectMultiple(attrs={"size": 5}),
        }

    def clean(self):
        cleaned_data = super().clean()
        self.validate_budget(cleaned_data)
        self.validate_actual_cost(cleaned_data)
        self.validate_team_members(cleaned_data)
        self.validate_status(cleaned_data)
        self.validate_priority(cleaned_data)
        return cleaned_data

    def validate_budget(self, cleaned_data):
        budget = cleaned_data.get("budget")
        if budget is not None and budget < 0:
            raise ValidationError("Budget must be a positive number or zero.")

    def validate_actual_cost(self, cleaned_data):
        budget = cleaned_data.get("budget")
        actual_cost = cleaned_data.get("actual_cost")
        if actual_cost is not None and budget is not None and actual_cost > budget:
            raise ValidationError("Actual cost cannot be greater than the budget.")

    def validate_team_members(self, cleaned_data):
        team_members = cleaned_data.get("team_members")
        if team_members:
            existing_members = set(
                Project.objects.values_list("team_members", flat=True)
            )
            for member in team_members:
                if member.pk not in existing_members:
                    raise ValidationError(
                        f"User '{member}' does not exist in the system."
                    )

    def validate_status(self, cleaned_data):
        status = cleaned_data.get("status")
        if status not in dict(Project.STATUS_CHOICES):
            raise ValidationError("Invalid status selected.")

    def validate_priority(self, cleaned_data):
        priority = cleaned_data.get("priority")
        if priority not in dict(Project.PRIORITY_CHOICES):
            raise ValidationError("Invalid priority selected.")

In [ ]:
from django import forms
from django.core.exceptions import ValidationError
from .models import TimeEntry
from users.models import UserProfile  # Import your user model
from tasks.models import Task  # Import your task model if available

class TimeEntryForm(forms.ModelForm):
    """
    Form for creating or editing time entries.
    """

    class Meta:
        model = TimeEntry
        fields = [
            "user",
            "task",
            "project",
            "start_time",
            "end_time",
            "hours_worked",
            "description",  # Optional fields
            "internal_notes",
            "is_billable",
        ]
        labels = {
            "start_time": "Start Time",
            "end_time": "End Time",
            "hours_worked": "Hours Worked",
            "description": "Description",
            "internal_notes": "Internal Notes",
            "is_billable": "Billable",
        }
        help_texts = {
            "user": "Select the user for this time entry.",
            "task": "Select the task related to this time entry (optional).",
            "project": "Select the project for which time is being logged.",
            "hours_worked": "Enter the number of hours worked on this time entry.",
        }
        required = [
            "user",
            "project",
            "start_time",
            "end_time",
            "hours_worked",
        ]

    def clean(self):
        cleaned_data = super().clean()
        self.validate_time_range(cleaned_data)
        self.validate_hours_worked(cleaned_data)
        return cleaned_data

    def validate_time_range(self, cleaned_data):
        start_time = cleaned_data.get("start_time")
        end_time = cleaned_data.get("end_time")
        if start_time and end_time and start_time >= end_time:
            raise ValidationError("End time must be after the start time.")

    def validate_hours_worked(self, cleaned_data):
        hours_worked = cleaned_data.get("hours_worked")
        if hours_worked is not None and hours_worked <= 0:
            raise ValidationError("Hours worked must be a positive number.")

        # Additional validation based on user-specific constraints
        user = cleaned_data.get("user")
        if user:
            max_daily_hours = user.profile.max_daily_hours
            if max_daily_hours and hours_worked > max_daily_hours:
                raise ValidationError(f"Hours worked exceeds the maximum daily limit for {user}.")

    def clean_task(self):
        task = self.cleaned_data.get("task")
        if task and not Task.objects.filter(pk=task.pk).exists():
            raise ValidationError("Selected task does not exist.")
        return task

    def clean_user(self):
        user = self.cleaned_data.get("user")
        if user and not UserProfile.objects.filter(pk=user.pk).exists():
            raise ValidationError("Selected user does not exist.")
        return user

    def clean_project(self):
        project = self.cleaned_data.get("project")
        if project and not Project.objects.filter(pk=project.pk).exists():
            raise ValidationError("Selected project does not exist.")
        return project


In [ ]:
from django import forms
from django.core.exceptions import ValidationError
from .models import Task

class TaskForm(forms.ModelForm):
    """
    Form for creating or editing tasks.
    """

    class Meta:
        model = Task
        fields = [
            "title",
            "description",
            "assignee",
            "due_date",
            "status",
            "priority",
            "project",
            "estimated_effort",
            "internal_notes",
            "dependent_tasks",
        ]
        labels = {
            "due_date": "Due Date",
            "estimated_effort": "Estimated Effort (hours)",
            "internal_notes": "Internal Notes",
            "dependent_tasks": "Dependent Tasks",
        }
        help_texts = {
            "title": "Enter a descriptive title for the task.",
            "status": "Select the current status of the task.",
            "priority": "Set the priority level of the task.",
            "project": "Select the project to which this task belongs.",
            "estimated_effort": "Enter the estimated effort required for this task in hours.",
            "internal_notes": "Add any internal notes or attachments related to this task.",
            "dependent_tasks": "Specify any tasks that must be completed before this one.",
        }
        required = {
            "assignee",
            "project",
            "title",
        }

    def clean(self):
        cleaned_data = super().clean()
        self.validate_due_date(cleaned_data)
        self.validate_estimated_effort(cleaned_data)
        self.validate_dependent_tasks(cleaned_data)
        return cleaned_data

    def validate_due_date(self, cleaned_data):
        due_date = cleaned_data.get("due_date")
        if due_date is not None and due_date < timezone.now().date():
            raise ValidationError("Due date cannot be in the past.")

    def validate_estimated_effort(self, cleaned_data):
        estimated_effort = cleaned_data.get("estimated_effort")
        if estimated_effort is not None and estimated_effort <= 0:
            raise ValidationError("Estimated effort must be a positive number.")

    def validate_dependent_tasks(self, cleaned_data):
        dependent_tasks = cleaned_data.get("dependent_tasks")
        project = cleaned_data.get("project")
        if dependent_tasks:
            for task in dependent_tasks:
                if task.project != project:
                    raise ValidationError(
                        "Dependent tasks must belong to the same project."
                    )


In [ ]:
from django import forms
from django.core.exceptions import ValidationError
from .models import Expense


class ExpenseForm(forms.ModelForm):
    """
    Form for creating or editing expenses.
    """

    class Meta:
        model = Expense
        fields = [
            "amount",
            "category",
            "description",
            "date",
            "project",
            "receipt_image",
            "vendor",
            "payment_method",
            "internal_notes",
        ]
        labels = {
            "date": "Expense Date",
            "receipt_image": "Receipt Image",
            "vendor": "Vendor",
            "payment_method": "Payment Method",
            "internal_notes": "Internal Notes",
        }
        help_texts = {
            "amount": "Enter the amount of the expense.",
            "category": "Specify the category of the expense.",
            "description": "Provide a description of the expense.",
            "project": "Select the project for which this expense is incurred.",
            "receipt_image": "Upload an image of the receipt for this expense.",
            "vendor": "Specify the vendor from whom the expense was incurred.",
            "payment_method": "Select the payment method used for this expense.",
            "internal_notes": "Add any internal notes or remarks for reference.",
        }

    def clean(self):
        cleaned_data = super().clean()
        self.validate_amount_against_budget(cleaned_data)
        return cleaned_data

    def validate_amount_against_budget(self, cleaned_data):
        project = cleaned_data.get("project")
        amount = cleaned_data.get("amount")
        if project:
            if amount is not None and amount > project.remaining_budget:
                raise ValidationError(
                    "Expense amount exceeds the remaining project budget."
                )

    def clean_vendor(self):
        vendor = self.cleaned_data.get("vendor")
        if vendor:
            # Implement custom validation logic here
            pass
        return vendor

In [ ]:
# Import necessary modules and classes
import logging
from django import forms
from django.contrib.auth.forms import (
    UserCreationForm,
    AuthenticationForm,
    PasswordResetForm,
)
from django.contrib.auth.tokens import default_token_generator
from django.contrib.sites.shortcuts import get_current_site
from django.core.mail import EmailMessage
from django.template.loader import render_to_string
from django.utils.encoding import force_bytes
from django.utils.http import urlsafe_base64_encode
from django.utils.translation import gettext_lazy as _
from .models import CustomUser

# Configure logging
logger = logging.getLogger(__name__)


# Define SignUpForm class
class SignUpForm(UserCreationForm):
    email = forms.EmailField(
        max_length=254, help_text=_("Required. Inform a valid email address.")
    )
    first_name = forms.CharField(max_length=30, label=_("First Name"))
    last_name = forms.CharField(max_length=30, label=_("Last Name"))

    class Meta:
        model = CustomUser
        fields = (
            "username",
            "email",
            "first_name",
            "last_name",
            "password1",
            "password2",
        )

    # Implement password strength validation logic
    def clean_password1(self):
        password1 = self.cleaned_data.get("password1")
        # Implement password strength validation logic here
        return password1

    # Include CSRF protection
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fields["csrfmiddlewaretoken"].widget = forms.HiddenInput()


# Define CustomPasswordResetForm class
class CustomPasswordResetForm(PasswordResetForm):
    # Implement advanced email validation
    def clean_email(self):
        email = self.cleaned_data.get("email")
        # Implement advanced email validation logic here
        return email

    # Add error handling for email delivery
    def send_mail(
        self,
        subject_template_name,
        email_template_name,
        context,
        from_email,
        to_email,
        html_email_template_name=None,
    ):
        try:
            # Send email
            super().send_mail(
                subject_template_name,
                email_template_name,
                context,
                from_email,
                to_email,
                html_email_template_name,
            )
        except Exception as e:
            # Handle email delivery failure
            logger.error(f"Failed to send email: {e}")
            # Log the error and/or display a user-friendly message
            pass

    # Log email events and potential errors
    def save(self, *args, **kwargs):
        # Log email events and potential errors
        logger.info("Password reset email sent successfully.")
        try:
            return super().save(*args, **kwargs)
        except Exception as e:
            logger.error(f"Error occurred while saving password reset form: {e}")
            raise

    # Provide customization for email templates
    def get_email_context(self, user, domain_override=None, extra_email_context=None):
        context = super().get_email_context(
            user,
            domain_override=domain_override,
            extra_email_context=extra_email_context,
        )
        # Customize email context as needed
        return context